# **<center>Practica Tensorflow TV</center>**
 

#### <center>13 de Diciembre, 2021</center>

## **Modelo Denoising a Resolver**

### 1. **Formulación del problema**

$$
\begin{equation}\label{eq:p1}
 u^* = \mbox{arg}\min_{u} C(u) = \mbox{arg}\min_{u}  \int_{\Omega} (f - u)^2 d\textbf{x} + \lambda TV_{\epsilon}(u) ~~~ (1)
\end{equation}
$$

donde:

$$
TV_{\epsilon}(u) = \int_{\Omega}| \nabla u|_{\epsilon} d\textbf{x} = \int_{\Omega}\sqrt{u_x^2 + u_y^2 + \epsilon} ~ d\textbf{x}
$$

Minimización con descenso de gradiente:

$$
u^{t+1} = u^t - \alpha \frac{\partial C(u)}{\partial u}
$$

### 2. **Práctica**

Se pide obtener la solución de (1) usando un framework (TensorFlow, Pytorch, etc) provisto de diferenciación automática.

In [ ]:
!git clone https://github.com/rollervan/FunMat.git temp
!cp -r ./temp/Codes/Images ./
!rm -rf ./temp

**Importamos librerias**

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm


**Definiciones**

In [ ]:
def Prior_p(u, p=1, eps = 1e-6):
#  Esta función devuelve el término apriori |∇u|^p
#  Completar
    ux, uy = 
    mod = 
    prior=  

    return prior


def gradient(u):
#  Esta función devuelve el gradiente de u
#  ux, uy = ∇u
#  Completar
    x_filter = 


    x_filter = tf.reshape(x_filter, [3, 3, 1, 1])


    y_filter = 
    
    y_filter = tf.reshape(y_filter, [3, 3, 1, 1])

    ux0 = tf.nn.conv2d(tf.expand_dims(u[:,:,:,0],-1), x_filter, strides=[1, 1, 1, 1], padding="SAME")
    ux1 = tf.nn.conv2d(tf.expand_dims(u[:,:,:,1],-1), x_filter, strides=[1, 1, 1, 1], padding="SAME")
    ux2 = tf.nn.conv2d(tf.expand_dims(u[:,:,:,2],-1), x_filter, strides=[1, 1, 1, 1], padding="SAME")
    ux = tf.concat([ux0,ux1,ux2],-1)
    uy0 = tf.nn.conv2d(tf.expand_dims(u[:,:,:,0],-1), y_filter, strides=[1, 1, 1, 1], padding="SAME")
    uy1 = tf.nn.conv2d(tf.expand_dims(u[:,:,:,1],-1), y_filter, strides=[1, 1, 1, 1], padding="SAME")
    uy2 = tf.nn.conv2d(tf.expand_dims(u[:,:,:,2],-1), y_filter, strides=[1, 1, 1, 1], padding="SAME")
    uy = tf.concat([uy0,uy1,uy2],-1)
    return ux, uy


**Cargamos, cambiamos el tamaño y normalizamos la imagen**

In [ ]:
IM_ROWS = 256
IM_COLS = 256
IM_DEPTH = 3

im = cv2.imread('Images/vaca.jpg')
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

im = cv2.resize(im,(IM_ROWS,IM_COLS))
im = np.reshape(im,newshape=[1,IM_ROWS,IM_COLS,IM_DEPTH])
im = cv2.normalize(im.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
im_org = im
im = im + 1*np.random.normal(0, 0.1,im.shape)
im = np.minimum(1.0,np.maximum(0.,im))

**Imagen Ruidosa**

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(im[0,...])
plt.show()

**Creamos variables en TensorFlow**

In [ ]:
# Completar

org = tf.Variable(tf.zeros(shape=(1,IM_ROWS,IM_COLS, IM_DEPTH)), name="org", trainable=???)
org.assign(im_org)
f = tf.Variable(tf.zeros(shape=(1,IM_ROWS,IM_COLS, IM_DEPTH)), name="f", trainable=???)
f.assign(im)
u = tf.Variable(tf.zeros(shape=(1,IM_ROWS,IM_COLS, IM_DEPTH)), name="u", trainable=???)
u.assign(im)

**Comprobamos la función 'gradient'**

In [ ]:
ux,uy = gradient(f)
mod = tf.sqrt( ux**2  + uy**2)
plt.imshow(mod[0,...])

In [ ]:
ux,uy = gradient(f)
fig, axs = plt.subplots(1,2,figsize=(20,20))
axs[0].imshow(ux[0,...])
axs[1].imshow(uy[0,...])

**Definimos la función de coste (funcional de energía) a minimizar y el gradiente asociado**

In [ ]:

lda = 1 #@param {type:"raw"}

def Cost(u):
    return  # Completar

def grad(u):
    with tf.GradientTape() as g:
        C = Cost(u)
        dC_du = g.gradient(C, u)
    return C, dC_du



**Seleccionamos el optimizador y creamos el bucle de minimización**

In [ ]:
# Reasignamos el valor original de la imagen ruidosa
u.assign(im)

train_C_results = []
psnr_results = []
opt =  # Completar
Nit = 1000 #@param {type:"raw"}

for i in tqdm(range(Nit)):

    C_value, grads = grad(u)
    opt.apply_gradients([(grads, u)])

    psnr_value = tf.image.psnr(org, u, max_val=1.0)
    # Track progress

    train_C_results.append(C_value)
    psnr_results.append(psnr_value)


In [ ]:
fig, axs = plt.subplots(1,2,figsize=(20,9))

axs[0].plot(train_C_results)
axs[0].set_title('Cost Value')
axs[0].set_xlabel('Iteraciones')
axs[1].plot(psnr_results)
axs[1].set_title('PSNR (db)')
axs[1].set_xlabel('Iteraciones')

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(20,20))
axs[0].imshow(f[0,...])
axs[1].imshow(u[0,...])